In [1]:
import pandas as pd
import time
import os
import csv

from selenium import webdriver
from selenium.webdriver.common.by import By
# from seleniumFunction import *
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse, urlunparse

# ==================== GLOBAL VARIABLES
# Set Chrome options:
chrome_options = Options()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox') 
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-application-cache")
chrome_options.add_argument("--disable-cache")
chrome_options.add_argument("--disk-cache-size=0")

In [2]:
%%capture
df = pd.read_csv('vietnam_accommodation.csv')

In [3]:
df.shape

(29446, 36)

In [4]:
# df.url.values[:100]

In [5]:
def url_edited(url):
    try:
        parsed_url = urlparse(url)
        new_path = "/vi-vn" + parsed_url.path
        return urlunparse(parsed_url._replace(path=new_path, query=""))
    except:
        return ""
        
def get_url(url):
    try:
        driver = webdriver.Chrome(chrome_options)
        driver.get(url)
        time.sleep(2)
        elem = driver.find_element(By.CSS_SELECTOR, "a[data-element-name='property-card-content']")
        href = elem.get_attribute('href')
        return url_edited(href)
    except:
        return ""       

In [12]:
get_url('https://www.agoda.com/partners/partnersearch.aspx?hid=73270')

'https://www.agoda.com/vi-vn/bamboo-sapa-hotel/hotel/sapa-vn.html'

In [60]:
for url in df.url.values[40:60]:
    print(get_url(url))

https://www.agoda.com/vi-vn/saigon-star-hotel/hotel/ho-chi-minh-city-vn.html
https://www.agoda.com/vi-vn/vien-dong-hotel/hotel/ho-chi-minh-city-vn.html
https://www.agoda.com/vi-vn/hai-yen-hotel/hotel/hoi-an-vn.html
https://www.agoda.com/vi-vn/hoi-an-beach-resort/hotel/hoi-an-vn.html
https://www.agoda.com/vi-vn/hoi-an-historic-hotel/hotel/hoi-an-vn.html
https://www.agoda.com/vi-vn/vinh-hung-2-city-hotel/hotel/hoi-an-vn.html
https://www.agoda.com/vi-vn/century-riverside-hue-hotel/hotel/hue-vn.html
https://www.agoda.com/vi-vn/saigon-morin-hotel/hotel/hue-vn.html
https://www.agoda.com/vi-vn/huong-giang-hotel-resort-spa/hotel/hue-vn.html
https://www.agoda.com/vi-vn/ana-mandara-cam-ranh_2/hotel/nha-trang-vn.html
https://www.agoda.com/vi-vn/nha-trang-lodge-hotel/hotel/nha-trang-vn.html
https://www.agoda.com/vi-vn/tran-vien-dong/hotel/nha-trang-vn.html
https://www.agoda.com/vi-vn/yasaka-saigon-resort-hotel-spa/hotel/nha-trang-vn.html
https://www.agoda.com/vi-vn/bamboo-village-beach-resort/hote

In [4]:
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from urllib.parse import urlparse, urlunparse
import time
import csv

# Define url_edited function
def url_edited(url):
    try:
        parsed_url = urlparse(url)
        new_path = "/vi-vn" + parsed_url.path
        return urlunparse(parsed_url._replace(path=new_path, query=""))
    except Exception as e:
        print(f"Error editing URL: {url} - {e}")
        return ""

# Define get_url function
def get_url(url):
    try:
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(url)
        time.sleep(2)  # Ensure the page loads
        elem = driver.find_element(By.CSS_SELECTOR, "a[data-element-name='property-card-content']")
        href = elem.get_attribute('href')
        driver.close()
        return url_edited(href)
    except Exception as e:
        print(f"Error fetching URL: {url}")
        return ""

# Initialize output files
with open("url_edited.csv", mode="w", newline='', encoding="utf-8") as edited_file, \
     open("crawl_again.csv", mode="w", newline='', encoding="utf-8") as error_file:

    # Define CSV writers
    edited_writer = csv.DictWriter(edited_file, fieldnames=["url", "url_edited"])
    error_writer = csv.DictWriter(error_file, fieldnames=["url"])
    
    # Write headers
    edited_writer.writeheader()
    error_writer.writeheader()

# Load your DataFrame (replace with actual data source)
# Example: df = pd.read_csv("input_file.csv")
# df = df.sample(n=100)

# Process each URL
for url in tqdm(df.url.values, desc="Processing URLs"):
    try:
        # Fetch the edited URL
        url_result = get_url(url)
        if url_result:  # Check if the result is not empty
            with open("url_edited.csv", mode="a", newline='', encoding="utf-8") as edited_file:
                edited_writer = csv.DictWriter(edited_file, fieldnames=["url", "url_edited"])
                edited_writer.writerow({"url": url, "url_edited": url_result})
        else:
            with open("crawl_again.csv", mode="a", newline='', encoding="utf-8") as error_file:
                error_writer = csv.DictWriter(error_file, fieldnames=["url"])
                error_writer.writerow({"url": url})
    except Exception as e:
        # Log the URL for retry if an exception occurs
        with open("crawl_again.csv", mode="a", newline='', encoding="utf-8") as error_file:
            error_writer = csv.DictWriter(error_file, fieldnames=["url"])
            error_writer.writerow({"url": url})
        print(f"Error processing URL: {url}")

print("Processing completed. Check 'url_edited.csv' for results and 'crawl_again.csv' for errors.")


PermissionError: [Errno 13] Permission denied: 'crawl_again.csv'